In [8]:
# Configuración del entorno
import sys
from pathlib import Path

# Agregar el directorio raíz al path para imports
root_dir = Path.cwd().parent.parent
if str(root_dir) not in sys.path:
    sys.path.insert(0, str(root_dir))

print(f"✅ Root directory agregado al path: {root_dir}")

✅ Root directory agregado al path: f:\GitHub\supply-chain-data-notebooks


# Contexto de Negocio y Marco de Trabajo

## Empresa y situación
Operación de retail nacional con múltiples WMS exportando CSV diarios de inventario y movimientos. Se requiere ingesta confiable al DWH para analítica.

## Qué / Por qué / Para qué / Cuándo / Cómo
- Qué: Ingesta batch de archivos CSV del WMS hacia tablas Parquet en el DWH.
- Por qué: Centralizar y estandarizar datos para consultas rápidas y consistentes.
- Para qué: Habilitar dashboards e informes operativos y de inventario.
- Cuándo: Procesos nocturnos y reintentos ante archivos tardíos.
- Cómo: Validación de esquema, enriquecimiento con fecha de ingesta y escritura en formato columna.

---
id: "DE-01"
title: "Batch ingestion from WMS CSV to Data Warehouse"
specialty: "Data Engineering"
process: "Deliver"
level: "Intermediate"
tags: ["etl", "warehouse", "inventory", "python", "sql"]
estimated_time_min: 45
---

## 🎯 Contexto del Notebook

### ¿Qué?
Ingesta batch de datos de inventario desde archivos CSV (simulando WMS) hacia un modelo analítico.

### ¿Por qué?
Los sistemas WMS generan archivos planos diarios. Consolidarlos en un warehouse permite análisis histórico, tendencias y alertas.

### ¿Para qué?
- Construir tablas de hechos de inventario
- Habilitar reportes de cobertura, rotación y obsolescencia
- Base para modelos de optimización de stock

### ¿Cuándo?
Ejecutar diariamente en horarios de baja operación (ej: 2 AM) post-cierre de turno WMS.

### ¿Cómo?
1. Leer CSV desde `data/raw/inventory.csv`
2. Validar esquema y tipos
3. Transformar (agregar timestamp, calcular métricas)
4. Escribir a `data/processed/` o conectar a DB

In [9]:
import pandas as pd
from pathlib import Path
from src.utils.paths import DATA_RAW, DATA_PROCESSED, ensure_dirs
from src.utils.logging import get_logger

logger = get_logger('DE-01')
ensure_dirs()
logger.info('Iniciando ingesta batch de inventario')
print('✅ Librerías cargadas y rutas preparadas')

2025-12-01 13:12:11,959 - DE-01 - INFO - Iniciando ingesta batch de inventario


✅ Librerías cargadas y rutas preparadas


In [10]:
# Leer CSV de inventario
df = pd.read_csv(DATA_RAW / 'inventory.csv')
logger.info(f'Registros cargados: {len(df)}')
print(df.head())
print(df.info())

2025-12-01 13:12:22,197 - DE-01 - INFO - Registros cargados: 3000


  location_id        sku  on_hand
0     LOC-001  SKU-00087       86
1     LOC-001  SKU-00145       38
2     LOC-001  SKU-00163       53
3     LOC-001  SKU-00024      108
4     LOC-001  SKU-00056       10
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   location_id  3000 non-null   object
 1   sku          3000 non-null   object
 2   on_hand      3000 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 70.4+ KB
None


In [11]:
# Transformaciones básicas
df['ingestion_date'] = pd.Timestamp.now()
df['on_hand_value'] = df['on_hand'] * 10  # mock: precio unitario
print(df[['location_id','sku','on_hand','on_hand_value']].head())

  location_id        sku  on_hand  on_hand_value
0     LOC-001  SKU-00087       86            860
1     LOC-001  SKU-00145       38            380
2     LOC-001  SKU-00163       53            530
3     LOC-001  SKU-00024      108           1080
4     LOC-001  SKU-00056       10            100


In [12]:
# Escribir a processed
output = DATA_PROCESSED / 'inventory_fact.parquet'
df.to_parquet(output, index=False)
logger.info(f'Datos escritos en {output}')
print('✅ Ingesta completada')

2025-12-01 13:12:42,190 - DE-01 - INFO - Datos escritos en F:\GitHub\supply-chain-data-notebooks\data\processed\inventory_fact.parquet


✅ Ingesta completada
